<a href="https://colab.research.google.com/github/adsl23589/flitwang/blob/master/YouTube_%E6%92%AD%E6%94%BE%E6%B8%85%E5%96%AE%E5%8C%AF%E5%87%BA%E5%B7%A5%E5%85%B7.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import tkinter as tk
from tkinter import messagebox, filedialog
import threading
import csv
import yt_dlp

class PlaylistExporterApp:
    def __init__(self, root):
        self.root = root
        self.root.title("YouTube 播放清單匯出工具")
        self.root.geometry("500x250")

        # 1. 說明標籤
        self.label_instruction = tk.Label(root, text="請貼上 YouTube 播放清單網址：", font=("微軟正黑體", 12))
        self.label_instruction.pack(pady=(20, 5))

        # 2. 輸入框
        self.entry_url = tk.Entry(root, width=50, font=("Arial", 10))
        self.entry_url.pack(pady=5)

        # 3. 狀態顯示區
        self.label_status = tk.Label(root, text="準備就緒", fg="gray", font=("微軟正黑體", 10))
        self.label_status.pack(pady=10)

        # 4. 按鈕區 (使用 Frame 來並排按鈕)
        self.button_frame = tk.Frame(root)
        self.button_frame.pack(pady=10)

        self.btn_txt = tk.Button(self.button_frame, text="匯出成文字檔 (.txt)",
                                 command=lambda: self.start_export("txt"),
                                 bg="#f0f0f0", font=("微軟正黑體", 10), width=20)
        self.btn_txt.pack(side=tk.LEFT, padx=10)

        self.btn_csv = tk.Button(self.button_frame, text="匯出成表格 (.csv)",
                                 command=lambda: self.start_export("csv"),
                                 bg="#f0f0f0", font=("微軟正黑體", 10), width=20)
        self.btn_csv.pack(side=tk.LEFT, padx=10)

    def update_status(self, message, color="blue"):
        """更新狀態標籤的文字與顏色"""
        self.label_status.config(text=message, fg=color)

    def toggle_buttons(self, state):
        """啟用或停用按鈕 (避免重複點擊)"""
        if state == "disabled":
            self.btn_txt.config(state=tk.DISABLED)
            self.btn_csv.config(state=tk.DISABLED)
        else:
            self.btn_txt.config(state=tk.NORMAL)
            self.btn_csv.config(state=tk.NORMAL)

    def start_export(self, format_type):
        """按鈕點擊後的觸發函數，啟動多執行緒"""
        url = self.entry_url.get().strip()

        # 基本驗證
        if not url:
            messagebox.showwarning("警告", "請輸入網址！")
            return
        if "youtube.com" not in url and "youtu.be" not in url:
            messagebox.showwarning("錯誤", "這看起來不像 YouTube 網址，請檢查後再試。")
            return

        # 鎖定按鈕，更新狀態
        self.toggle_buttons("disabled")
        self.update_status("正在讀取清單資訊，請稍候...", "orange")

        # 開啟新執行緒去跑爬蟲，避免視窗卡死
        thread = threading.Thread(target=self.run_extraction, args=(url, format_type))
        thread.start()

    def run_extraction(self, url, format_type):
        """實際執行爬蟲的邏輯 (在背景執行)"""
        ydl_opts = {
            'extract_flat': True,  # 關鍵：只抓標題不下載影片，速度快
            'ignoreerrors': True,  # 忽略無法讀取的私人影片
            'quiet': True,         # 減少終端機輸出
        }

        try:
            with yt_dlp.YoutubeDL(ydl_opts) as ydl:
                info = ydl.extract_info(url, download=False)

                if 'entries' not in info:
                    raise ValueError("無法找到播放清單內容，請確認網址是否正確。")

                # 資料整理
                playlist_data = []
                for index, entry in enumerate(info['entries'], start=1):
                    if entry: # 確保該條目存在（避免已刪除影片）
                        title = entry.get('title', '無標題')
                        # 處理網址：如果是短網址或 ID，組合成完整網址
                        video_url = entry.get('url', '')
                        if 'youtube.com' not in video_url:
                            video_url = f"https://www.youtube.com/watch?v={entry.get('id')}"

                        playlist_data.append({
                            'index': index,
                            'title': title,
                            'url': video_url
                        })

            # 抓取完成，通知主執行緒進行存檔
            self.root.after(0, self.ask_save_file, playlist_data, format_type)

        except Exception as e:
            error_msg = str(e)
            self.root.after(0, self.handle_error, error_msg)

    def handle_error(self, error_msg):
        """處理錯誤狀況"""
        self.update_status("發生錯誤", "red")
        self.toggle_buttons("normal")
        messagebox.showerror("錯誤", f"讀取失敗：\n{error_msg}")

    def ask_save_file(self, data, format_type):
        """彈出存檔視窗並寫入檔案"""
        self.update_status("讀取完成，正在存檔...", "green")

        if format_type == "txt":
            file_path = filedialog.asksaveasfilename(
                defaultextension=".txt",
                filetypes=[("Text file", "*.txt")],
                initialfile="playlist_export.txt",
                title="儲存文字檔"
            )
            if file_path:
                self.save_txt(file_path, data)
            else:
                self.update_status("已取消存檔", "gray")
                self.toggle_buttons("normal")

        elif format_type == "csv":
            file_path = filedialog.asksaveasfilename(
                defaultextension=".csv",
                filetypes=[("CSV file", "*.csv")],
                initialfile="playlist_export.csv",
                title="儲存表格檔"
            )
            if file_path:
                self.save_csv(file_path, data)
            else:
                self.update_status("已取消存檔", "gray")
                self.toggle_buttons("normal")

    def save_txt(self, path, data):
        try:
            with open(path, 'w', encoding='utf-8') as f:
                for item in data:
                    line = f"{item['index']}. {item['title']} - {item['url']}\n"
                    f.write(line)

            messagebox.showinfo("成功", "檔案匯出成功！")
            self.update_status("匯出成功！", "green")
        except Exception as e:
            messagebox.showerror("存檔失敗", str(e))
        finally:
            self.toggle_buttons("normal")

    def save_csv(self, path, data):
        try:
            # newline='' 是為了防止 CSV 多出空白行
            with open(path, 'w', encoding='utf-8-sig', newline='') as f:
                writer = csv.writer(f)
                # 寫入標頭
                writer.writerow(['編號', '影片標題', '影片連結'])
                # 寫入內容
                for item in data:
                    writer.writerow([item['index'], item['title'], item['url']])

            messagebox.showinfo("成功", "檔案匯出成功！")
            self.update_status("匯出成功！", "green")
        except Exception as e:
            messagebox.showerror("存檔失敗", str(e))
        finally:
            self.toggle_buttons("normal")

if __name__ == "__main__":
    root = tk.Tk()
    app = PlaylistExporterApp(root)
    root.mainloop()